In [81]:
import logging
logger = logging.getLogger('requests_throttler')
logger.addHandler(logging.NullHandler())
logger.propagate = False

import numpy as np
import pandas as pd
from pandas import datetime
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
%matplotlib inline

In [109]:
def model_holidays(df):
    
    state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
    school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

    state = pd.DataFrame({'holiday': 'state_holiday',
                          'ds': pd.to_datetime(state_dates)})
    school = pd.DataFrame({'holiday': 'school_holiday',
                          'ds': pd.to_datetime(school_dates)})

    holidays = pd.concat((state, school))
    return holidays

def forecast_eval(actual, forecast):
    metric_df = forecast.set_index('ds')[['yhat']].join(actual.set_index('ds').y).reset_index()
    metric_df.dropna(inplace=True)
    return np.sqrt(mean_squared_error(metric_df.y, metric_df.yhat))

def forecast_sale(storeNumber, df, holidays):
    
    sales = df[df.Store == storeNumber].loc[:, ['Date', 'Sales']]
    sales = sales.sort_index(ascending = False)
    sales['Date'] = pd.DatetimeIndex(sales['Date'])
    sales = sales.rename(columns = {'Date': 'ds',
                                    'Sales': 'y'})
    
    my_model = Prophet(holidays = holidays, yearly_seasonality = 70)
    my_model.fit(sales)
    
    future_dates = my_model.make_future_dataframe(periods = 7*7, freq='W')
    forecast = my_model.predict(future_dates)
    rmse = forecast_eval(sales, forecast)
    return ( storeNumber, my_model, forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']], rmse )

In [85]:
df = pd.read_csv("train.csv", low_memory = False)
df = df[(df["Open"] != 0) & (df['Sales'] != 0)]

test = pd.read_csv("test.csv", low_memory = False)

train_holidays = model_holidays(df)
test_holidays = model_holidays(test)
total_holidays = pd.concat((train_holidays, test_holidays)).drop_duplicates().reset_index(drop=True)

In [60]:
total_holidays.head()

,ds,holiday
0,2015-06-04,state_holiday
1,2015-05-25,state_holiday
2,2015-05-14,state_holiday
3,2015-05-01,state_holiday
4,2015-01-06,state_holiday


In [113]:
# all_stores =  list(xrange(1115))
# selected_stores = np.random.choice(all_stores, 20, replace=False)
selected_stores = pd.read_csv('storeEval.csv')
all_forecasts = []

for store in selected_stores['Store']:
    f = forecast_sale(store, df, total_holidays)
    all_forecasts.append(f)

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True t

INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet.forecaster:Disabling daily seasonality. Run prophet with daily_seasonality=True t

In [114]:
len(all_forecasts)

100

In [116]:
d = []
for forecast in all_forecasts:
    d.append({'Store': forecast[0], 'RMSE': forecast[3]})
store_eval = pd.DataFrame(d)

In [117]:
store_eval.to_csv('storeEval.csv', sep=',', encoding='utf-8')